<!-- #CS529 - Selected Tocpics in Computer Vision -->
<!-- #CS336 - Advanced Computer Vision -->
<!-- #CS231 - Computer Vision Project-->
#Background Subtraction and Replacement with Google DeepLabv3+
17520324 - Nguyen Thanh Danh</br>
<!-- MAIN -->
Description:</br>
This project is a combination of two modules:
+ Segmentation module: we included four pretrained Google DeepLabv3+ models for users to choose from, these models have the ability to segment twenty kinds of objects apart from background.
+ Changing background module: this part allows user to choose a new image as a background and apply one segmented object to that background.

Main Reference Paper and Source Code:
+ L.-C. Chen, Y. Zhu, G. Papandreou, F. Schroff, and H. Adam, ''Encoder-decoder with atrous separable convolution for semantic image segmentation,'' in ECCV, 2018.
+ https://github.com/tensorflow/models/tree/master/research/deeplab

<!-- Publish online try at: https://colab.research.google.com/drive/1DRsAPsLNtxOljWY6RIbp4aKdJSMYz1PI?fbclid=IwAR1kN2Z9lznPhNUCQ84qCpQ0561FNMXGfa8p3iQaKE4wgpK020qXbCK_Y2U#sandboxMode=true&scrollTo=jhC_IGzuNZJb -->

#1. Segmentation Module - Google DeepLabv3+ pretrained model

In [ ]:
#@title Choose pretrained DeepLabv3+ model:

#Import Libraries
import os
from io import BytesIO
import tarfile
import tempfile
from six.moves import urllib

from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
from PIL import Image

import tensorflow as tf

#DeepLabModel definition
class DeepLabModel(object):
  """Class to load deeplab model and run inference."""

  INPUT_TENSOR_NAME = 'ImageTensor:0'
  OUTPUT_TENSOR_NAME = 'SemanticPredictions:0'
  INPUT_SIZE = 513
  FROZEN_GRAPH_NAME = 'frozen_inference_graph'

  def __init__(self, tarball_path):
    """Creates and loads pretrained deeplab model."""
    self.graph = tf.Graph()

    graph_def = None
    # Extract frozen graph from tar archive.
    tar_file = tarfile.open(tarball_path)
    for tar_info in tar_file.getmembers():
      if self.FROZEN_GRAPH_NAME in os.path.basename(tar_info.name):
        file_handle = tar_file.extractfile(tar_info)
        graph_def = tf.compat.v1.GraphDef.FromString(file_handle.read()) #change code due to tf 2.0
        break

    tar_file.close()

    if graph_def is None:
      raise RuntimeError('Cannot find inference graph in tar archive.')

    with self.graph.as_default():
      tf.import_graph_def(graph_def, name='')

    self.sess = tf.compat.v1.Session(graph=self.graph) #change code due to tf 2.0

  def run(self, image):
    """Runs inference on a single image.

    Args:
      image: A PIL.Image object, raw input image.

    Returns:
      resized_image: RGB image resized from original input image.
      seg_map: Segmentation map of `resized_image`.
    """
    width, height = image.size
    resize_ratio = 1.0 * self.INPUT_SIZE / max(width, height)
    target_size = (int(resize_ratio * width), int(resize_ratio * height))
    resized_image = image.convert('RGB').resize(target_size, Image.ANTIALIAS)
    batch_seg_map = self.sess.run(
        self.OUTPUT_TENSOR_NAME,
        feed_dict={self.INPUT_TENSOR_NAME: [np.asarray(resized_image)]})
    seg_map = batch_seg_map[0]
    return resized_image, seg_map



def create_pascal_label_colormap():
  """Creates a label colormap used in PASCAL VOC segmentation benchmark.

  Returns:
    A Colormap for visualizing segmentation results.
  """
  colormap = np.zeros((256, 3), dtype=int)
  ind = np.arange(256, dtype=int)

  for shift in reversed(range(8)):
    for channel in range(3):
      colormap[:, channel] |= ((ind >> channel) & 1) << shift
    ind >>= 3

  return colormap


def label_to_color_image(label):
  """Adds color defined by the dataset colormap to the label.

  Args:
    label: A 2D array with integer type, storing the segmentation label.

  Returns:
    result: A 2D array with floating type. The element of the array
      is the color indexed by the corresponding element in the input label
      to the PASCAL color map.

  Raises:
    ValueError: If label is not of rank 2 or its value is larger than color
      map maximum entry.
  """
  if label.ndim != 2:
    raise ValueError('Expect 2-D input label')

  colormap = create_pascal_label_colormap()

  if np.max(label) >= len(colormap):
    raise ValueError('label value too large.')

  return colormap[label]


def vis_segmentation(image, seg_map):
  """Visualizes input image, segmentation map and overlay view."""
  plt.figure(figsize=(15, 5))
  grid_spec = gridspec.GridSpec(1, 4, width_ratios=[6, 6, 6, 1])

  plt.subplot(grid_spec[0])
  plt.imshow(image)
  plt.axis('off')
  plt.title('input image')

  plt.subplot(grid_spec[1])
  seg_image = label_to_color_image(seg_map).astype(np.uint8)
  plt.imshow(seg_image)
  plt.axis('off')
  plt.title('segmentation map')

  plt.subplot(grid_spec[2])
  plt.imshow(image)
  plt.imshow(seg_image, alpha=0.7)
  plt.axis('off')
  plt.title('segmentation overlay')

  unique_labels = np.unique(seg_map)
  ax = plt.subplot(grid_spec[3])
  plt.imshow(FULL_COLOR_MAP[unique_labels].astype(np.uint8), interpolation='nearest')
  ax.yaxis.tick_right()
  plt.yticks(range(len(unique_labels)), LABEL_NAMES[unique_labels])
  plt.xticks([], [])
  ax.tick_params(width=0.0)
  plt.grid('off')
  plt.show()


LABEL_NAMES = np.asarray([
    'background', 'aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus',
    'car', 'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike',
    'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tv'])

FULL_LABEL_MAP = np.arange(len(LABEL_NAMES)).reshape(len(LABEL_NAMES), 1)
FULL_COLOR_MAP = label_to_color_image(FULL_LABEL_MAP)



#Load pretrained model - 4 models to choose
MODEL_NAME = 'xception_coco_voctrainval'  # @param ['mobilenetv2_coco_voctrainaug', 'mobilenetv2_coco_voctrainval', 'xception_coco_voctrainaug', 'xception_coco_voctrainval']

_DOWNLOAD_URL_PREFIX = 'http://download.tensorflow.org/models/'
_MODEL_URLS = {
    'mobilenetv2_coco_voctrainaug':
        'deeplabv3_mnv2_pascal_train_aug_2018_01_29.tar.gz',
    'mobilenetv2_coco_voctrainval':
        'deeplabv3_mnv2_pascal_trainval_2018_01_29.tar.gz',
    'xception_coco_voctrainaug':
        'deeplabv3_pascal_train_aug_2018_01_04.tar.gz',
    'xception_coco_voctrainval':
        'deeplabv3_pascal_trainval_2018_01_04.tar.gz',
}
_TARBALL_NAME = 'deeplab_model.tar.gz'

model_dir = tempfile.mkdtemp()
tf.io.gfile.GFile(model_dir) #change code due to tf 2.0

download_path = os.path.join(model_dir, _TARBALL_NAME)
print('downloading model, this might take a while...')
urllib.request.urlretrieve(_DOWNLOAD_URL_PREFIX + _MODEL_URLS[MODEL_NAME],
                   download_path)
print('download completed! loading DeepLab model...')

MODEL = DeepLabModel(download_path)
print('model loaded successfully!')

downloading model, this might take a while...
download completed! loading DeepLab model...
model loaded successfully!


In [ ]:
#@title Input image to segment:
IMAGE_URL = 'https://scx2.b-cdn.net/gfx/news/hires/2018/2-dog.jpg'  #@param {type:"string"}

def run_visualization(url):
  """Inferences DeepLab model and visualizes result."""
  try:
    f = urllib.request.urlopen(url)
    jpeg_str = f.read()
    original_im = Image.open(BytesIO(jpeg_str))
  except IOError:
    print('Cannot retrieve image. Please check url: ' + url)
    return

  print('running deeplab on image %s...' % url)
  resized_im, seg_map = MODEL.run(original_im)

  vis_segmentation(resized_im, seg_map)
  
  return original_im, resized_im, seg_map #

image_url = IMAGE_URL
original_im, resized_im, seg_map = run_visualization(image_url) #

#2. Changing background module

In [ ]:
#@title Choose what segmented object to reject:
Choose_what_to_reject = 'background' #@param [ 'background', 'aeroplane','bicycle', 'bird', 'boat', 'bottle', 'bus', 'car', 'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike', 'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tv']



#Post-process on seg_map to change background
#define dictionary for labels names
Labels_dict = {
    0: 'background',
    1: 'aeroplane',
    2: 'bicycle',
    3: 'bird',
    4: 'boat',
    5: 'bottle',
    6: 'bus',
    7: 'car',
    8: 'cat',
    9: 'chair',
    10: 'cow',
    11: 'diningtable',
    12: 'dog',
    13: 'horse',
    14: 'motorbike',
    15: 'person',
    16: 'pottedplant',
    17: 'sheep',
    18: 'sofa',
    19: 'train',
    20: 'tv',
}
# print(Labels_dict)

#show what segmap contents
# print('unique values of segmap: ', np.unique(seg_map))
param_of_what_to_reject = []
for i in np.unique(seg_map):
  # print(LABEL_NAMES[i])
  param_of_what_to_reject.append(LABEL_NAMES[i])

# for i in range(len(LABEL_NAMES)):
#   print(' \'' + str(LABEL_NAMES[i])+'\': '+str(i)+',')


encode_object_to_reject = list(Labels_dict.keys())[list(Labels_dict.values()).index(Choose_what_to_reject)]
# print('encode of object to reject: ',encode_object_to_reject)
print('Chose', Choose_what_to_reject, 'as region(s) to mix!')

Chose background as region(s) to mix!


In [ ]:
#@title Input new background image:

IMAGE_URL2 = 'https://i.pinimg.com/originals/da/bb/45/dabb4501ed0069a6082aa3459ccd1508.jpg'  #@param {type:"string"}

try:
  f2 = urllib.request.urlopen(IMAGE_URL2)
  jpeg_str2 = f2.read()
  bg_im = Image.open(BytesIO(jpeg_str2))
except IOError:
  print('Cannot retrieve image. Please check url: ' + IMAGE_URL2)

# plt.imshow(bg_im.resize((seg_map.shape[1],seg_map.shape[0])))
bg_im_resized = bg_im.resize((seg_map.shape[1],seg_map.shape[0]))
plt.title('New Background Image')
plt.imshow(bg_im_resized)

In [ ]:
#@title Result:
#Show mask (seg_map with 0 and 1)
mask = np.copy(seg_map)
newmask = np.copy(seg_map)
mask[newmask != encode_object_to_reject] = 1
mask[newmask == encode_object_to_reject] = 0
# print(np.unique(mask))
# print(np.unique(seg_map))
# plt.imshow(mask)



#show segmented object only
original_im_cp = np.array(resized_im)
for i in range(3):
  original_im_cp[:,:,i] = np.multiply(original_im_cp[:,:,i],mask)
# plt.imshow(original_im_cp)



#Show type of background image and original image
# print(type(bg_im))
# print(type(original_im_cp))
bg_im_resized = np.array(bg_im_resized)
# bg_im2 = np.resize(bg_im,(seg_map.shape[0],seg_map.shape[1],3))
# print(bg_im_resized.shape)
# print(original_im_cp.shape)


#Clear bg image where object locates
mask2 = np.copy(seg_map)
newmask2 = np.copy(seg_map)
mask2[newmask2 == encode_object_to_reject] = 1
mask2[newmask2 != encode_object_to_reject] = 0
# print(np.unique(mask2))

bg_im_resized_cp = np.copy(bg_im_resized)

for i in range(3):
  bg_im_resized_cp[:,:,i] = np.multiply(bg_im_resized[:,:,i],mask2)
plt.imshow(bg_im_resized_cp)
# print(np.unique(seg_map))


#Show result image
result = np.add(original_im_cp,bg_im_resized_cp)
plt.title('Result Image After Effect')
plt.imshow(result)

